In [61]:
from bs4 import BeautifulSoup
import requests
import re
import pprint
import wordfreq
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import pandas as pd
import random
import datetime
from sklearn.cluster import SpectralClustering

These functions are meant to extract presidential speeches from the site millercenter.org.  get_president(N) finds the Nth president and returns a "speeches" variable that is a list containing the text of each speech by that president.  The president's number is 1-indexed, so George Washington is president number 1.  This convention is followed throughout, except where otherwise specified.

In [62]:
#N = 8396
N = 1

# The 1st president (Washington) is actually numbered 44 on millercenter.org.  Thus, we have to convert
# the president's number 1 through 44 into a number suitable for millercenter.org.  Trump is, randomly enough
# president number 8396.
# The 22nd and 24th president are ordinarily considered to be Cleveland, Cleveland is only counted once.
# #24 is considered to be Harrison; that means there are only 44 presidents
def pres_numbers_list():
    return [44, 45] + [3, 4] + [141] + list(range(6, 44)) + [8396]

def get_pres_name(N):
    return ["George Washington", "John Adams", "Thomas Jefferson", "James Madison", "James Monroe", "John Quincy Adams",\
           "Andrew Jackson", "Martin van Buren", "William Harrison", "John Tyler", "James K. Polk", "Zachary Taylor",\
           "Millard Fillmore", "Franklin Pierce", "James Buchanan", "Abraham Lincoln", "Andrew Johnson",\
           "Ulysses S. Grant", "Rutherford B. Hayes", "James A. Garfield", "Chester A. Arthur", "Grover Cleveland",\
           "Benjamin Harrison", "William McKinley", "Theodore Roosevelt", "William Taft", "Woodrow Wilson",\
           "Warren G. Harding", "Calvin Coolidge", "Herbert Hoover", "Franklin D. Roosevelt", "Harry S. Truman",\
           "Dwight D. Eisenhower", "John F. Kennedy", "Lyndon B. Johnson", "Richard Nixon", "Gerald Ford",\
           "Jimmy Carter", "Ronald Reagan", "George H. W. Bush", "Bill Clinton", "George W. Bush", "Barack Obama",\
           "Donald Trump"][N-1]

def get_president(N):
    if N < 1 or N > 44:
        print("Error: no president", N)
        return list()
    N = pres_numbers_list()[N - 1]
    millerpage = f"https://millercenter.org/the-presidency/presidential-speeches?field_president_target_id[{N}]={N}"
    page = requests.get(millerpage)
    soup = BeautifulSoup(page.content, 'html.parser')
    #dummy = 'a href="/the-presidency/presidential-speeches'
    speechlist = soup.find_all(href=re.compile('/the-presidency/presidential-speeches/'))
    URLlist = ["https://millercenter.org" + x['href'] for x in speechlist]
    #pprint.pprint(URLlist)
    speeches = list()
    n = -1
    for URL in URLlist:
        n = n + 1
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        for e in soup.find_all('br'):
            e.replace_with('\n')
        x = soup('h3', text="Transcript")
        listofps = x[0].parent.findChildren('p')
        #print([type(p.contents[0]) for p in listofps])
        #string = string.replace(u'\xa0', u' ')
        # print([p.contents[0].name for p in listofps])
        textofspeech = " ".join([" ".join([c for c in p.contents if not c.name]) for p in listofps])
        # Replace funny single quote #8217 with single quote #39
        textofspeech = textofspeech.replace('\’', "'")
        textofspeech = textofspeech.replace("\'", "'")
        speeches.append(textofspeech)
    return speeches


get_overrep_words takes a list of the text of each speech, and returns a list of "overrepresented words" the form:

[(word1, weight1), (word2, weight2), ...]

The "weight" for each word is calculated as follows.  The goal is to have a weight that reflects (1) the relative incidence of the word in the speech corpus compared with its usual incidence in the English language, (2) the overall incidence of the word in the speech corpus.  Thus, a word that is highly unusual ("supercalifragilisticexpealidocious") but only appears once in the speech corpus is not that interesting.  In contrast, a word that is very common, but appears only the usual amount in the corpus ("the") is also not interesting.  The weight for each words is computed as a product of these two factors.

Getting into the details: To compute the weight, the incidence count of the word is first tallied in the combined speeches of a president.
Using the wordfreq library, each word's frequency in English is obtained (actually the log of the frequency), so that it can be compared with the log of the frequency in the speeches.
Using the statsmodels library, a least squares regression is used to find the linear relationship between these two logs.

This linear relationship can then be used to compute the predicted English frequency for a word that has the given corpus frequency.  This predicted English frequency can be compared to the word's actual English frequency.  The difference between predicted and actual (using subtraction) is essentially a measure of how much more common the word is in the speech corpus compared to the English language.  A weight value of 1 would mean that the word is "e" times more common in the speech corpus as compared with English.

Next, the number of words in each set of speeches can range from 20,000 to 800,000, so the number of appearances is multiplied by (800,000 / length) to get the number that would appear in 800,000.  Then, the log is taken.  This number is multiplied by the weight to get the final weight value.  This takes into account that we care more about words that appear many times as opposed to only once.

In [63]:
def get_overrep_words(speeches):
    speeches_combined = " ".join(speeches)
    print("Length of speeches:", len(speeches_combined))
    wordlist = wordfreq.tokenize(speeches_combined, 'en')
    length = len(wordlist)
    freqlist = [(x, wordlist.count(x)) for x in set(wordlist)]
    freqlist.sort(key = lambda x: x[1])
    freqmap = [(np.log(wordfreq.word_frequency(x[0], 'en')), np.log(x[1] / length)) for x in freqlist]
    xy = list(zip(*freqmap))
    #plt.figure()
    #plt.plot(*xy, 'b.')
    dfxy = pd.DataFrame(xy).T
    dfxy.columns = ['logen', 'logcorpus']
    dfxy = dfxy.replace([np.inf, -np.inf], np.nan).dropna()
    #print(dfxy)
    model = sm.OLS.from_formula('logen ~ logcorpus', dfxy) 
    regr = model.fit()
    #regr.summary()

    # Replace funny single quote #8217 with single quote #39
    freqlist = [(x[0].replace('’', "'"), x[1]) for x in freqlist]
    
    weightlist = [(x[0], (regr.predict(pd.Series([np.log(x[1] / length)], name='logcorpus')).iloc[0] - \
            np.log(wordfreq.word_frequency(x[0], 'en'))) * np.log(x[1] * 800000 / length) \
            ) for x in freqlist if ((x[1] * 800000 / length > 1) and x[0] != 'applause' and x[0] != 'laughter')]
    
    weightlist.sort(key = lambda x: x[1])

    return [x for x in weightlist if (x[1] > 1 and not np.isinf(x[1]))], freqlist

In [64]:
def make_two_speech_lists(speeches):
    lst1 = np.random.choice(speeches, size=int(len(speeches)/2), replace=False)
    lst2 = [x for x in speeches if not (x in lst1)]
    return (lst1, lst2)

get_paired_words returns a list of the form:

[(word1, sim1), (word2, sim2), ...]

Where the similarity score (sim1) for word 1 is the minimum of the word's weight for president M and for president N.  Only words that appear in both president M and president N's speeches will get a score.

get_similarity finds the total similarity score by adding them up.

The result is a similarity score that compares the two presidents.  If the presidents share more words in common (and with higher weights), then their similarity score will tend to be higher.

In [65]:
def get_paired_words(overrep_words, M, N):
    if M < 1 or M > len(overrep_words) or N < 1 or N > len(overrep_words):
        print("M or N is out of bounds in get_paired_words.")
        return list()
    paired_words = list()
    for x in overrep_words[M-1]:
        for y in overrep_words[N-1]:
            if x[0] == y[0]:
                paired_words.append((x[0], min(x[1],y[1])))
    return paired_words

def get_similarity(overrep_words, M, N):
    pw = get_paired_words(overrep_words, M, N)
    total = sum(x[1] for x in pw)
    return total

This code block loops through a range of presidents, getting a list of overrepresented words (with their weights) and a list of frequencies for each.

In [67]:
NUMP = 44

train_test = True

random.seed(datetime.datetime.now())

def generate_overrep_words(NUMP):

    overrep_words = list()
    overrep_words_test = list()

    for N in range(45 - NUMP, 45):
        print(get_pres_name(N))
        speeches = get_president(N)
        sp_train, sp_test = make_two_speech_lists(speeches)
        if train_test:
            if len(sp_train) == 0:
                sp_train = sp_test.copy()
                print("Only one (testing) speech for president ", N)
            if len(sp_test) == 0:
                sp_test = sp_train.copy()
                print("Only one (training) speech for president ", N)
            ov, fr = get_overrep_words(sp_train)
            ov_test, fr_test = get_overrep_words(sp_test)
            overrep_words.append(ov)
            overrep_words_test.append(ov_test)
        else:
            ov, fr = get_overrep_words(speeches)
            overrep_words.append(ov)
            
    return overrep_words, overrep_words_test
    
overrep_words, overrep_words_test = generate_overrep_words(NUMP)

George Washington
Length of speeches: 72076


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log


Length of speeches: 56566
John Adams
Length of speeches: 44879
Length of speeches: 44030
Thomas Jefferson
Length of speeches: 65222
Length of speeches: 51395
James Madison
Length of speeches: 68480
Length of speeches: 68343
James Monroe
Length of speeches: 152366
Length of speeches: 144855
John Quincy Adams
Length of speeches: 35248
Length of speeches: 186625
Andrew Jackson
Length of speeches: 271600
Length of speeches: 236650
Martin van Buren
Length of speeches: 201093
Length of speeches: 190105
William Harrison
Only one (testing) speech for president  9
Length of speeches: 49777
Length of speeches: 49777
John Tyler
Length of speeches: 225523
Length of speeches: 51347
James K. Polk
Length of speeches: 245738
Length of speeches: 79753
Zachary Taylor
Length of speeches: 12422
Length of speeches: 56131
Millard Fillmore
Length of speeches: 132366
Length of speeches: 102888
Franklin Pierce
Length of speeches: 210014
Length of speeches: 96427
James Buchanan
Length of speeches: 161502
Length

Create a similarity matrix such that X[i,j] is the similarity of presidents i and j.  Then, print the cluster labels and cluster centers.  Then use SpectralClustering to perform the clustering.

In [68]:
def similarity_matrix(overrep_words):
    X = np.zeros((NUMP, NUMP))
    for i in range(0, NUMP):
        for j in range(0, NUMP):
            X[i,j] = get_similarity(overrep_words, i+1, j+1)
    return X

n_clusters = 3

X = similarity_matrix(overrep_words)
#clustering = AgglomerativeClustering(n_clusters=n_clusters, affinity="precomputed", linkage="average").fit(X)
clustering = SpectralClustering(n_clusters=n_clusters, affinity="precomputed", assign_labels="discretize").fit(X)
print("Labels:", clustering.labels_)
#print("Centers:", clustering.cluster_centers_)

if train_test:
    X_test = similarity_matrix(overrep_words_test)
    #clustering_test = AgglomerativeClustering(n_clusters=n_clusters, affinity="precomputed", linkage="average").fit(X_test)    
    clustering_test = SpectralClustering(n_clusters=n_clusters, affinity="precomputed", assign_labels="discretize").fit(X_test)
    print("Labels test:", clustering_test.labels_)

Labels: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 2 2 2 2 2 2 0 2 2 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]
Labels test: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2]


compute_cluster_overlap will compute the overlap between the training and testing data.  The overlap is defined by assuming that each cluster in the training data corresponds to the cluster(s) in the testing data with the best fit (the maximum overlap.)  Then, the number of presidents whose training cluster matches their testing cluster is counted.

In [70]:
def compute_cluster_overlap(labels1, labels2):
    if len(labels1) != len(labels2):
        print("Error in compute_cluster_overlap: labels1, labels2 not same length.")
        return
    corresp = dict()
    for lab1 in set(labels1):
        ct = dict()
        for lab2 in set(labels2):
            for i in range(len(labels1)):
                ct.setdefault(lab2, 0)
                if labels1[i] == lab1 and labels2[i] == lab2:
                    ct[lab2] += 1
        ctreverse={v:k for k,v in ct.items()}
        corresp[lab1] = ctreverse[max(ctreverse)]
    total = 0
    for i in range(len(labels1)):
        if labels2[i] == corresp[labels1[i]]:
            total += 1
    print("Correspondence (training cluster: testing cluster):", corresp)
    print("Overlap ratio:", total, "out of", len(labels1))        
    return total, len(labels1)

compute_cluster_overlap(clustering.labels_, clustering_test.labels_)

Correspondence (training cluster: testing cluster): {0: 2, 1: 0, 2: 1}
Overlap ratio: 41 out of 44


(41, 44)

These functions make use of only the training set data (or all of the data, if you have set train_test = False).

get_inlist will make a list of the numbers of the president that belong to the ith cluster in the training set (zero indexed this time.)

get_characteristic_words accepts as an argument a list of president numbers (zero indexed this time.)  Thus, a list [0, 2, 3] will combine the 0th, 2nd, and 3rd presidents into a group, which is then compared with all presidents not in the list.  You can use get_inlist to generate this list from a cluster number, or you can pick a single president with a single element list like [2].

The function finds the median weight in the "inlist" and the median weight in the "outlist," subtracting the two to find the score for each word.  In the resulting list, a positive score indicates that a word has a high weight in the inlist, while a negative score indicates that it has a high weight in the outlist.  If there are an odd number of presidents in the list, the median takes the lower of the two middle numbers.

Thus, to find the most characteristic words in the inlist, scroll to the end of the output and look at the words with the highest score

In [76]:
import statistics

def medlow(lst):
    sortlst = sorted(lst)
    if len(sortlst) % 2 == 0:
        return(sortlst[int(len(sortlst) / 2) - 1])
    else:
        return(sortlst[int(len(sortlst) / 2 - 0.5)])

def get_inlist(lab, i):
    return [j for j in range(0, len(lab)) if lab[j] == i]
    
def get_characteristic_words(overrep_words, inlist):
    wordscore_in = dict()
    wordscore_out = dict()
    for k in range(0, len(overrep_words)):
        if k in inlist:
            for word, score in overrep_words[k]:
                wordscore_in.setdefault(word, list())
                wordscore_in[word].append(score)
        else:
            for word, score in overrep_words[k]:
                wordscore_out.setdefault(word, list())
                wordscore_out[word].append(score)
    for word in wordscore_in.keys():
        for i in range(0, len(inlist) - len(wordscore_in[word])):
            wordscore_in[word].append(0)
    for word in wordscore_out.keys():
        for i in range(0, (len(overrep_words) - len(inlist)) - len(wordscore_out[word])):
            wordscore_out[word].append(0)
    scorelist = list()
    for word in wordscore_in.keys():
        med_in = medlow(wordscore_in[word]) if word in wordscore_in.keys() else 0
        med_out = medlow(wordscore_out[word]) if word in wordscore_out.keys() else 0
        scorelist.append((word, med_in - med_out))
    return sorted(scorelist, key = lambda x: x[1])

print(get_characteristic_words(overrep_words, get_inlist(clustering.labels_, 0)))

[('expenditures', -14.94857613014275), ('upon', -14.158713141372049), ('treasury', -13.580561691197962), ('justly', -13.536569441663827), ('appropriations', -13.448385610485207), ('guaranty', -12.070039776518698), ('earnestly', -12.051287331332), ('receipts', -11.874896716692994), ('gratifying', -11.187271051770413), ('consideration', -11.079334361374785), ('evils', -10.962352012690564), ('duties', -10.841934284917995), ('commerce', -10.814249864694753), ('appropriation', -10.738449788428527), ('intercourse', -10.541506552010645), ('treaty', -10.335885834443344), ('hitherto', -10.248836317039254), ('constitution', -10.18086725159676), ('expediency', -9.926967277351139), ('practicable', -9.450824511826774), ('states', -9.078328244234083), ('duty', -9.018487555896687), ('satisfactory', -8.62550218080027), ('labors', -8.561636396207232), ('pecuniary', -8.506881310171734), ('necessity', -8.311887310679223), ('authorized', -8.107746711477931), ('tranquillity', -7.975209648016353), ('interes

In [72]:
print(get_characteristic_words(overrep_words, get_inlist(clustering.labels_, 1)))

[('nation', -8.194668290040918), ('legislation', -7.249040327097415), ('america', -7.1036187810236795), ('american', -7.061417546567518), ('allies', -6.574428957968351), ('progress', -6.096393165426474), ('our', -5.508569109640246), ('freedom', -5.321358612373296), ('peace', -5.088947287369555), ('burdens', -5.067838390948353), ('abroad', -4.957376772228166), ("nation's", -4.867961525356991), ('enact', -4.3915122210721), ('strengthen', -3.9833253218946045), ('responsibility', -3.901355854279635), ('railroads', -3.743761661378633), ('peril', -3.711889576713235), ('millions', -3.672852343509745), ('adequate', -3.556941266739866), ('nations', -3.5022532726595133), ('toward', -3.301871668367711), ('aggression', -3.238087000608994), ('determination', -3.098500272246476), ('tyranny', -3.0108296318323786), ('plainly', -2.955764606576195), ('hemisphere', -2.8381585361731014), ('secure', -2.7357446352908945), ('strengthening', -2.703308428665845), ('renew', -2.6616596035998072), ('prosperous', 

In [73]:
print(get_characteristic_words(overrep_words, get_inlist(clustering.labels_, 2)))

[('peace', -5.557560039486593), ('entertained', -4.275263658284931), ('cherish', -4.105840900778211), ('our', -3.967269859300676), ('nation', -3.897435373582776), ('solemn', -3.64787252194116), ('countrymen', -3.6226797465467255), ('faithfully', -2.8559847689616324), ('liberty', -2.826501373874554), ('wisdom', -2.804821546281878), ('preserve', -2.7820895334219706), ("nation's", -2.725923517030613), ('millions', -2.6046788026787153), ('vigorous', -2.4931015696816305), ('patriotism', -2.3260074863220193), ('constitution', -2.1743005057670457), ('equal', -2.113776670464859), ('negotiation', -2.048865424864617), ('freedom', -1.8932190073402764), ('principle', -1.84361197414305), ('shall', -1.7881360131716653), ('gratitude', -1.7366164254962881), ('war', -1.6684297490089142), ('communicated', -1.531806042562858), ('prudent', -1.485063852972946), ('principles', -1.3929387886261577), ('citizen', -1.265702079227036), ('dangers', -1.2369030019588223), ('country', -1.205940533978466), ('confiden

In [74]:
print(get_characteristic_words(overrep_words, [43])) # Trump

[('congress', -8.552905601607097), ('states', -6.116482826675825), ('peace', -5.080916304021876), ('duties', -3.7778616081313814), ('legislation', -2.4768540107676102), ('wisdom', -1.4452312336534952), ('duty', -1.4403132162184935), ('constitution', -1.2571313324797355), ('senate', -0.9086194698324368), ('united', -0.799708153188357), ('interests', -0.5833570500435217), ('abroad', 0.18932772388278551), ('patriotic', 0.43373537471768353), ('gratitude', 0.6913536473241488), ('citizen', 0.7157105856478063), ('kindness', 1.0092409193454195), ('sustain', 1.0092409193454195), ('bullets', 1.0092409193454195), ('whatsoever', 1.0092409193454195), ('frontier', 1.0092409193454195), ('reduces', 1.0092409193454195), ('arbitrary', 1.0092409193454195), ('terminal', 1.0330461033080158), ('guards', 1.0330461033080158), ('colleagues', 1.0330461033080158), ("hadn't", 1.0330461033080158), ('promised', 1.0468459360756917), ('texas', 1.0491494046222438), ('fellow', 1.0491494046222438), ('plants', 1.04914940

In [75]:
print(get_characteristic_words(overrep_words, [42])) # Obama

[('citizens', -7.173959404639206), ('commerce', -4.40454851756846), ('nations', -3.7258912061161173), ('states', -2.0759297483701165), ('legislation', -1.3942245571106553), ('patriotism', -1.2227237597385194), ('united', -1.1203139729599307), ('prosperity', -0.3235115830734312), ('departments', -0.04791120953779404), ('congress', 0.22495164302099724), ('citizen', 0.3321043635133927), ('relations', 0.3330292608742169), ('constitutional', 0.37821884701409614), ('confidence', 0.5838620435587294), ('path', 1.0043760391893974), ('smart', 1.0043760391893974), ('obstacles', 1.0188164442102754), ('dioxide', 1.0188164442102754), ('denying', 1.0188164442102754), ('marines', 1.0188164442102754), ('yemen', 1.0188164442102754), ('excuses', 1.0188164442102754), ('deployment', 1.0188164442102754), ('drought', 1.0188164442102754), ('growing', 1.0344722338682222), ('home', 1.0458358640683665), ('manufacturers', 1.0683962254240378), ('shoulder', 1.0825502025132558), ('reaching', 1.0825502025132558), ('a